In [14]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans

# Paths and categories
categories = ["HealthySpiral", "HealthyMeander", "PatientSpiral", "PatientMeander"]

X_images = []      # list to store image data for CNN
X_features = []    # list to store extracted feature vectors
initial_labels = []  # healthy/patient label (optional, for analysis)

# Loop over each category folder
for category in categories:
    folder_path = category
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            # Load image in color and grayscale
            img_color = cv2.imread(img_path)  # BGR color image
            gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
            # Resize images to a standard size (e.g., 224x224) for consistency
            img_color = cv2.resize(img_color, (224, 224))
            gray = cv2.resize(gray, (224, 224))
            # Append to image list (for CNN) and extract features for feature list
            X_images.append(img_color)
            features = []  # will be filled by feature extraction
            
            # Feature 1: Size (normalized area of drawing)
            # Invert image to get drawing in white on black for contour detection
            _, bw = cv2.threshold(cv2.bitwise_not(gray), 128, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) == 0:
                # If no contour found (blank image), skip or append zero features
                features = [0, 0, 0, 0, 0]
            else:
                cnt = max(contours, key=cv2.contourArea)  # largest contour
                area = cv2.contourArea(cnt)
                norm_area = area / float(gray.shape[0] * gray.shape[1])
                # Feature 2: Stroke length (contour perimeter normalized)
                perimeter = cv2.arcLength(cnt, closed=True)
                norm_perimeter = perimeter / float(gray.shape[0] * gray.shape[1])
                # Compute angles along the contour to measure smoothness/tremor
                angles = []
                for i in range(len(cnt) - 1):
                    p1 = cnt[i][0]
                    p2 = cnt[i+1][0]
                    dx = p2[0] - p1[0]
                    dy = p2[1] - p1[1]
                    angles.append(np.arctan2(dy, dx))
                angles = np.unwrap(angles)  # unwrap angles to avoid discontinuities
                if len(angles) >= 2:
                    angle_diffs = np.diff(angles)
                else:
                    angle_diffs = np.array([0.0])
                # Feature 3: Smoothness (mean absolute change in angle)
                smoothness = float(np.mean(np.abs(angle_diffs)))
                # Feature 4: Tremor (variability of angle changes)
                tremor = float(np.std(angle_diffs))
                # Feature 5: Number of stroke segments (number of contours)
                stroke_count = len(contours)
                features = [norm_area, norm_perimeter, smoothness, tremor, stroke_count]
            X_features.append(features)
            # Record initial group label (not severity, just healthy/patient) if needed
            initial_labels.append(0 if "Healthy" in category else 1)

# Convert lists to numpy arrays
X_images = np.array(X_images)         # shape (N, 224, 224, 3)
X_features = np.array(X_features)     # shape (N, feature_dim)
initial_labels = np.array(initial_labels)

# Apply K-Means clustering to features to derive 5 severity clusters (0–4)
kmeans = KMeans(n_clusters=5, random_state=42)
severity_labels = kmeans.fit_predict(X_features)  # cluster assignment for each sample

# Optionally, map clusters to severity scale by ordering cluster centroids 
# (e.g., cluster with most healthy samples -> 0, most extreme tremor -> 4).
# For simplicity, we use the cluster indices directly as severity labels.
print("Derived cluster centers (feature space):", kmeans.cluster_centers_)
print("Assigned severity label counts:", np.bincount(severity_labels))

Derived cluster centers (feature space): [[8.20579783e-02 4.52367605e-02 8.78108369e-01 9.20882751e-01
  4.56386293e+00]
 [3.90545281e-02 4.62636752e-02 9.09406029e-01 9.76526408e-01
  1.20500000e+02]
 [5.41301437e-02 3.53160399e-02 8.56267489e-01 8.95567030e-01
  4.14090909e+01]
 [6.85996969e-02 4.43187102e-02 8.79246636e-01 9.27294871e-01
  1.71824818e+01]
 [2.27281220e-02 2.27012175e-02 8.67535856e-01 9.19010494e-01
  7.29375000e+01]]
Assigned severity label counts: [321  10  44 137  16]


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Split data into train and test sets (e.g., 80% train, 20% test)
X_feat_train, X_feat_test, X_img_train, X_img_test, y_train, y_test = train_test_split(
    X_features, X_images, severity_labels, test_size=0.2, random_state=42, stratify=severity_labels
)

# --- k-Nearest Neighbors (k-NN) ---
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_feat_train, y_train)
y_pred_knn = knn.predict(X_feat_test)

print("k-NN Classification Accuracy (test): {:.2f}%".format(100 * accuracy_score(y_test, y_pred_knn)))
print("k-NN Classification Report:\n", classification_report(y_test, y_pred_knn))

# --- Support Vector Machine (SVM) ---
svm = SVC(kernel='rbf', C=1.0, probability=True, random_state=42)
svm.fit(X_feat_train, y_train)
y_pred_svm = svm.predict(X_feat_test)

print("SVM Classification Accuracy (test): {:.2f}%".format(100 * accuracy_score(y_test, y_pred_svm)))
print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))

# (Optional) Hyperparameter tuning for SVM using cross-validation
# from sklearn.model_selection import GridSearchCV
# param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly']}
# grid = GridSearchCV(SVC(probability=True), param_grid, cv=5)
# grid.fit(X_feat_train, y_train)
# best_svm = grid.best_estimator_
# print("Best SVM parameters:", grid.best_params_)

k-NN Classification Accuracy (test): 99.06%
k-NN Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        64
           1       1.00      0.50      0.67         2
           2       1.00      1.00      1.00         9
           3       1.00      1.00      1.00        28
           4       0.75      1.00      0.86         3

    accuracy                           0.99       106
   macro avg       0.95      0.90      0.90       106
weighted avg       0.99      0.99      0.99       106

SVM Classification Accuracy (test): 96.23%
SVM Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        64
           1       1.00      1.00      1.00         2
           2       0.86      0.67      0.75         9
           3       0.90      1.00      0.95        28
           4       1.00      0.67      0.80         3

    accuracy                           0.

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Normalize image pixel values to [0,1]
X_img_train = X_img_train.astype('float32') / 255.0
X_img_test  = X_img_test.astype('float32')  / 255.0

# Simple CNN model definition
model_cnn = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.GlobalAveragePooling2D(),  # reduce feature maps to a single 128-d vector
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='softmax')  # 5 severity classes
])
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = model_cnn.fit(X_img_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

# Evaluate on test set
test_loss, test_acc = model_cnn.evaluate(X_img_test, y_test, verbose=0)
print("CNN Test Accuracy: {:.2f}%".format(test_acc * 100))

Epoch 1/20


/opt/homebrew/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 369ms/step - accuracy: 0.4810 - loss: 1.3677 - val_accuracy: 0.6512 - val_loss: 0.9319
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 358ms/step - accuracy: 0.6026 - loss: 1.1083 - val_accuracy: 0.6512 - val_loss: 0.9150
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 380ms/step - accuracy: 0.5934 - loss: 1.1094 - val_accuracy: 0.6512 - val_loss: 0.9080
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 343ms/step - accuracy: 0.5974 - loss: 1.1432 - val_accuracy: 0.6512 - val_loss: 0.8947
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 362ms/step - accuracy: 0.6042 - loss: 1.0733 - val_accuracy: 0.6512 - val_loss: 0.8961
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 394ms/step - accuracy: 0.5953 - loss: 1.0689 - val_accuracy: 0.6512 - val_loss: 0.9279
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 347ms/step - accuracy: 0.6200 - loss: 1.0579 - val_accuracy: 0.6512 - val_loss: 0.9000
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 345ms/step - accuracy: 0.6142 - loss: 1.0626 - val_accuracy: 0.6512 - val_

In [17]:
# Transfer Learning with ResNet50
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # freeze the convolutional base
# Add custom top layers
x = layers.GlobalAveragePooling2D()(base_model.output)
output_layer = layers.Dense(5, activation='softmax')(x)
model_resnet = tf.keras.Model(inputs=base_model.input, outputs=output_layer)

model_resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train only the top layer for a few epochs
model_resnet.fit(X_img_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

# Fine-tune: unfreeze some layers of ResNet (e.g., last few blocks) and continue training with a smaller learning rate
base_model.trainable = True
# Freeze all layers except the last 50 layers (for example)
for layer in base_model.layers[:-50]:
    layer.trainable = False
# Recompile with a lower learning rate for fine-tuning
model_resnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                     loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_resnet.fit(X_img_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

# Evaluate ResNet model on test set
resnet_loss, resnet_acc = model_resnet.evaluate(X_img_test, y_test, verbose=0)
print("ResNet50 Transfer Learning Test Accuracy: {:.2f}%".format(resnet_acc * 100))

Epoch 1/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 777ms/step - accuracy: 0.2563 - loss: 1.4527 - val_accuracy: 0.6512 - val_loss: 0.9321
Epoch 2/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 704ms/step - accuracy: 0.5814 - loss: 1.1046 - val_accuracy: 0.6512 - val_loss: 0.8681
Epoch 3/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 790ms/step - accuracy: 0.5884 - loss: 1.0412 - val_accuracy: 0.6512 - val_loss: 0.8356
Epoch 4/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 714ms/step - accuracy: 0.6401 - loss: 0.9057 - val_accuracy: 0.7209 - val_loss: 0.8428
Epoch 5/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 699ms/step - accuracy: 0.5911 - loss: 1.0448 - val_accuracy: 0.6512 - val_loss: 0.8578
Epoch 1/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.3284 - loss: 1.5762 - val_accuracy: 0.6512 - val_loss: 0.8528
Epoch 2/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.7038 - loss: 0.7314 - val_accuracy: 0.6512 - val_loss: 0.8533
Epoch 3/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.7708 - loss: 0.6115 - val_accuracy: 0.6512 - val_lo

In [18]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# --- Method 1: Average probabilities from SVM and CNN (late fusion ensemble) ---
# Get probability predictions on the test set from SVM and CNN
svm_probs = svm.predict_proba(X_feat_test)             # shape (n_samples, 5)
cnn_probs = model_resnet.predict(X_img_test)           # shape (n_samples, 5)
avg_probs = (svm_probs + cnn_probs) / 2
ensemble_pred_labels = np.argmax(avg_probs, axis=1)

# --- Method 2: Feature fusion and new classifier (early fusion ensemble) ---
# Extract CNN embeddings (penultimate layer outputs) for train and test images
# For the ResNet model, we already have a global pooling layer output of size 2048 (before Dense).
# We can create a new model that outputs that layer (x) from model_resnet.
embed_model = tf.keras.Model(inputs=model_resnet.input, outputs=base_model.output)  # get conv base output (7x7x2048)
# Actually, better to get the pooled output directly:
embed_model = tf.keras.Model(inputs=model_resnet.input, outputs=model_resnet.layers[-2].output)  # output of GlobalAveragePooling2D
train_embeds = embed_model.predict(X_img_train)  # shape (n_train, 2048)
test_embeds = embed_model.predict(X_img_test)    # shape (n_test, 2048)

# Concatenate manual features with CNN embeddings
combined_train_features = np.concatenate([X_feat_train, train_embeds], axis=1)
combined_test_features  = np.concatenate([X_feat_test,  test_embeds],  axis=1)
print("Combined feature vector size:", combined_train_features.shape[1])

# Train an ensemble classifier on the combined features (e.g., Random Forest)
ensemble_clf = RandomForestClassifier(n_estimators=100, random_state=42)
ensemble_clf.fit(combined_train_features, y_train)
ensemble_pred = ensemble_clf.predict(combined_test_features)

# Evaluate the hybrid model
ensemble_acc = accuracy_score(y_test, ensemble_pred)
print("Ensemble (Feature+CNN) Accuracy (test): {:.2f}%".format(ensemble_acc * 100))
print("Ensemble Classification Report:\n", classification_report(y_test, ensemble_pred))

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 700ms/step


14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 641ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 488ms/step
Combined feature vector size: 2053
Ensemble (Feature+CNN) Accuracy (test): 80.19%
Ensemble Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.98      0.91        64
           1       0.00      0.00      0.00         2
           2       0.57      0.44      0.50         9
           3       0.72      0.64      0.68        28
           4       0.00      0.00      0.00         3

    accuracy                           0.80       106
   macro avg       0.43      0.41      0.42       106
weighted avg       0.75      0.80      0.77       106



/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
from sklearn.metrics import confusion_matrix

# Confusion matrix for ensemble predictions
cm = confusion_matrix(y_test, ensemble_pred)
print("Confusion Matrix (rows=true, cols=pred):\n", cm)

# Calculate sensitivity (recall) and specificity for each class
num_classes = cm.shape[0]
sensitivity = np.zeros(num_classes)
specificity = np.zeros(num_classes)
for i in range(num_classes):
    TP = cm[i, i]
    FN = np.sum(cm[i, :]) - TP
    FP = np.sum(cm[:, i]) - TP
    TN = np.sum(cm) - (TP + FP + FN)
    sensitivity[i] = TP / float(TP + FN) if (TP+FN) > 0 else 0.0
    specificity[i] = TN / float(TN + FP) if (TN+FP) > 0 else 0.0

print("Sensitivity (Recall) for classes 0-4:", np.round(sensitivity, 3))
print("Specificity for classes 0-4:", np.round(specificity, 3))

Confusion Matrix (rows=true, cols=pred):
 [[63  0  0  1  0]
 [ 0  0  0  2  0]
 [ 1  0  4  4  0]
 [10  0  0 18  0]
 [ 0  0  3  0  0]]
Sensitivity (Recall) for classes 0-4: [0.984 0.    0.444 0.643 0.   ]
Specificity for classes 0-4: [0.738 1.    0.969 0.91  1.   ]


In [20]:
import joblib
# Save models to disk
joblib.dump(svm, "svm_feature_model.pkl")
joblib.dump(ensemble_clf, "hybrid_ensemble_model.pkl")
model_resnet.save("cnn_severity_model.h5")

# Later, load the models (in deployment environment)
svm_loaded = joblib.load("svm_feature_model.pkl")
ensemble_loaded = joblib.load("hybrid_ensemble_model.pkl")
cnn_loaded = tf.keras.models.load_model("cnn_severity_model.h5")
# Recreate the embedding model for CNN (to get penultimate layer output if needed for hybrid)
embed_loaded = tf.keras.Model(inputs=cnn_loaded.input, outputs=cnn_loaded.layers[-2].output)

In [27]:
def predict_severity(image_path):
    # 1. Load and preprocess the image
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    img_resized = cv2.resize(img, (224, 224))
    gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    # 2. Extract features from the image
    _, bw = cv2.threshold(cv2.bitwise_not(gray), 128, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(cnt)
        norm_area = area / float(gray.shape[0] * gray.shape[1])
        perimeter = cv2.arcLength(cnt, True)
        norm_perimeter = perimeter / float(gray.shape[0] * gray.shape[1])
        angles = []
        for i in range(len(cnt) - 1):
            p1 = cnt[i][0]; p2 = cnt[i+1][0]
            angles.append(np.arctan2(p2[1]-p1[1], p2[0]-p1[0]))
        angles = np.unwrap(angles)
        angle_diffs = np.diff(angles) if len(angles) >= 2 else np.array([0.0])
        smoothness = float(np.mean(np.abs(angle_diffs))) if angle_diffs.size > 0 else 0.0
        tremor = float(np.std(angle_diffs)) if angle_diffs.size > 0 else 0.0
        stroke_count = len(contours)
        feat_vector = np.array([norm_area, norm_perimeter, smoothness, tremor, stroke_count]).reshape(1, -1)
    else:
        # if no contour found, use zeros
        feat_vector = np.zeros((1,5), dtype=float)
    # 3. Feature model prediction
    feat_pred_proba = svm_loaded.predict_proba(feat_vector)  # probability from SVM
    # 4. CNN model prediction
    img_input = img_resized.astype('float32')/255.0
    img_input = np.expand_dims(img_input, axis=0)  # shape (1,224,224,3)
    cnn_pred_proba = cnn_loaded.predict(img_input)
    # 5. Combine predictions - average probabilities
    avg_proba = (feat_pred_proba + cnn_pred_proba) / 2
    final_class = int(np.argmax(avg_proba, axis=1)[0])
    # Alternatively, use the hybrid model directly on combined features:
    # embed_feat = embed_loaded.predict(img_input)  # get embedding from CNN
    # combined_feat = np.concatenate([feat_vector, embed_feat], axis=1)
    # final_class = int(ensemble_loaded.predict(combined_feat)[0])
    return final_class

# Example usage:
print("Predicted severity:", predict_severity("mea3-P6.jpg"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
Predicted severity: 3
